# Import Required Libraries

In [9]:
from bs4 import BeautifulSoup
from urllib.request import urlopen, Request
import pandas as pd

import snowflake.connector as snow
from snowflake.connector.pandas_tools import write_pandas

import sqlalchemy as sql
import os

## Connection with Snowflake

In [73]:
conn = snow.connect(user="PBVIRANI",
   password="Patel@8732",
   account="pd06844.ap-south-1.aws",
   warehouse="DATALOADS",
   database="POC_DATASET",
   schema="POC_SCHEMA")


# Create a cursor object.
cur = conn.cursor()


## Load Data in Stock_fundamental Table

In [91]:
df = pd.read_excel('company.xlsx')
stock_id = 1
for i in df['Company']:
    SCRIP = i
    link = f'https://www.screener.in/company/{SCRIP}'
    hdr = {'User-Agent':'Mozilla/5.0'}
    req = Request(link,headers=hdr)

    try:
        page=urlopen(req)
        soup = BeautifulSoup(page)

        div_html = soup.find('div',{'class': 'company-ratios'})
        ul_html = div_html.find('ul',{'id': 'top-ratios'})
        market_cap = 0.0
        dis = {}    
        for li in ul_html.find_all("li"):
            name_span = li.find('span',{'class':'name'})
            num_span = li.find('span',{'class':'nowrap value'})
            num_span = num_span.text.replace(',', '')
            num_span = num_span.replace(' ', '')
            num_span = num_span.replace('\n', '')
            num_span = num_span.replace('₹', '')
            num_span = num_span.replace('Cr.', '')
            num_span = num_span.replace('C', '')
            num_span = num_span.replace('%', '')
            dis[name_span.text.strip()] = num_span if (num_span != '') else 0.0

        dis['stock_name'] = soup.find('h1',{'class':'margin-0 show-from-tablet-landscape'}).text
        section_html = soup.find('section',{'id': 'peers'})
        div_html = section_html.find('div')
        sub_div = div_html.find('p',{'class':'sub'}).text.strip()
        sub_div = sub_div.replace('\n\n','@')
        sub_div = sub_div.replace(' ','')
        sub_div = sub_div.split('@')
        dis['sector'] = sub_div[0].replace('\n','').split(":")[1]
        dis['industry'] = sub_div[1].replace('\n','').split(":")[1]
        dis['symbol'] = SCRIP
        cur.execute("INSERT INTO stock_fundamental(stock_id,stock_name,stock_symbol,pe,roce,book_value,roe,sector,industry,dividend_yield) VALUES("+str(stock_id)+",'"+dis['stock_name']+"','"+dis['symbol']+"',"+str(dis['Stock P/E'])+","+dis['ROCE']+","+dis['Book Value']+","+dis['ROE']+",'"+dis['sector']+"','"+dis['industry']+"',"+dis['Dividend Yield']+")")
        print(f'MARKET CAPITILIZATION - {SCRIP}: {dis}\n')
        stock_id += 1
    except:
        print(f'EXCEPTION THROWN: UNABLE TO FETCH DATA FOR {SCRIP}')
    

MARKET CAPITILIZATION - TCS: {'Market Cap': '1221334', 'Current Price': '3337', 'High / Low': '3836/2926', 'Stock P/E': '31.2', 'Book Value': '231', 'Dividend Yield': '1.29', 'ROCE': '61.3', 'ROE': '50.2', 'Face Value': '1.00', 'stock_name': 'Tata Consultancy Services Ltd', 'sector': 'IT-Software', 'industry': 'Computers-Software-Large', 'symbol': 'TCS'}

MARKET CAPITILIZATION - HCLTECH: {'Market Cap': '293225', 'Current Price': '1080', 'High / Low': '1215/876', 'Stock P/E': '25.2', 'Book Value': '149', 'Dividend Yield': '4.42', 'ROCE': '30.2', 'ROE': '25.0', 'Face Value': '2.00', 'stock_name': 'HCL Technologies Ltd', 'sector': 'IT-Software', 'industry': 'Computers-Software-Large', 'symbol': 'HCLTECH'}

MARKET CAPITILIZATION - WIPRO: {'Market Cap': '213178', 'Current Price': '388', 'High / Low': '616/372', 'Stock P/E': '17.2', 'Book Value': '107', 'Dividend Yield': '1.53', 'ROCE': '27.1', 'ROE': '24.4', 'Face Value': '2.00', 'stock_name': 'Wipro Ltd', 'sector': 'IT-Software', 'industry

In [92]:
#cur.execute("INSERT INTO stock_fundamental(stock_id,stock_name,stock_symbol,pe,roce,book_value,roe,sector,industry,dividend_yield) VALUES("+str(stock_id)+",'"+dis['stock_name']+"','"+dis['symbol']+"',"+dis['Stock P/E']+","+dis['ROCE']+","+dis['Book Value']+","+dis['ROE']+",'"+dis['sector']+"','"+dis['industry']+"',"+dis['Dividend Yield']+")")

## Load Data in SHAREHOLDING_PATTERN Table

In [174]:
cur.execute("SELECT stock_id,stock_name,stock_symbol FROM stock_fundamental")
one_row = cur.fetchall()
for i in one_row:
    shareholding = {'Promoters':0,'FIIs':0,'DIIs':0,'Government':0,'Public':0,'Others':0}
    stock = {}
    SCRIP = i[2]
    stock_name = i[1]
    stock_id =i[0]
    link = f'https://www.screener.in/company/{SCRIP}'
    hdr = {'User-Agent':'Mozilla/5.0'}
    req = Request(link,headers=hdr)
    column = ""
    try:
        page=urlopen(req)
        soup = BeautifulSoup(page)

        section_html = soup.find('section',{'id': 'shareholding'})
        div_html = section_html.find('div',{'class': 'responsive-holder'})
        table_html = div_html.find('table',{'class': 'data-table'})
        thead_html = table_html.find('thead')
        trow_html = thead_html.find('tr')
        column = trow_html.text
        column = column.strip().split('\n\n')
        column = column[0].split('\n')
#         print(column)
        tbody_html = table_html.find('tbody')
        for trow_html in tbody_html.find_all("tr"):
            temp = []
            #trow_html = tbody_html.find('tr')
            ttitle_html = trow_html('td',{'class':'text'})

            title = trow_html.text
            title = title.strip().split('\n\n')
            header = title[0]
            header = header.replace('+','')
            header = header.replace('\n','')
#             print(header.strip()) 
            for i in range(1,len(title)):
                if title[i].split('\n') != "":
                    temp = title[i].split()
            stock[header.strip()] = temp
        print(stock_name)
        df =pd.DataFrame (stock, columns = list(stock.keys()),index = column)
        print(df)
        print('---------------------------------------------------------------------------')
        for j in range(0,len(df['Promoters'])):
            if 'Promoters' in df: shareholding['Promoters'] = df['Promoters'][j]
            if 'FIIs' in df: shareholding['FIIs'] = df['FIIs'][j]
            if 'DIIs' in df: shareholding['DIIs'] = df['DIIs'][j]
            if 'Government' in df: shareholding['Government'] = df['Government'][j]
            if 'Public' in df: shareholding['Public'] = df['Public'][j]
            if 'Others' in df: shareholding['Others'] = df['Others'][j]
            cur.execute("INSERT INTO SHAREHOLDING_PATTERN(stock_id,stock_name,stock_symbol,quarter_year,promotoer,fiis,diis,government,public,other) VALUES("+str(stock_id)+",'"+stock_name+"','"+SCRIP+"','"+column[j]+"','"+str(shareholding['Promoters'])+"','"+str(shareholding['FIIs'])+"','"+str(shareholding['DIIs'])+"','"+str(shareholding['Government'])+"','"+str(shareholding['Public'])+"','"+str(shareholding['Others'])+"')")

    except:
        print(f'EXCEPTION THROWN: UNABLE TO FETCH DATA FOR {SCRIP}')

    

Tata Consultancy Services Ltd
         Promoters   FIIs  DIIs Government Public
Mar 2020     72.05  15.74  8.00       0.06   4.14
Jun 2020     72.05  15.85  8.02       0.07   4.01
Sep 2020     72.05  16.00  7.85       0.07   4.03
Dec 2020     72.05  15.88  7.71       0.07   4.29
Mar 2021     72.19  15.62  7.83       0.06   4.31
Jun 2021     72.19  15.43  7.97       0.04   4.38
Sep 2021     72.19  15.37  7.89       0.04   4.51
Dec 2021     72.19  14.98  7.93       0.04   4.86
Mar 2022     72.30  14.15  7.81       0.04   5.70
Jun 2022     72.30  13.50  8.36       0.05   5.80
Sep 2022     72.30  13.05  8.53       0.05   6.06
Dec 2022     72.30  12.94  9.25       0.05   5.45
---------------------------------------------------------------------------
HCL Technologies Ltd
         Promoters   FIIs   DIIs Government Public Others
Mar 2020     60.33  26.42   9.11       0.04   4.10   0.00
Jun 2020     60.33  25.62  10.01       0.04   4.00   0.00
Sep 2020     60.33  24.92  10.62       0.04   4.0

## Load Data in Cash_flow Table

In [183]:
cur.execute("SELECT stock_id,stock_name,stock_symbol FROM stock_fundamental")
one_row = cur.fetchall()

for i in one_row:
    cashflow = {'Cash from Operating Activity':0,'Cash from Investing Activity':0,'Cash from Financing Activity':0,'Net Cash Flow':0}
    stock = {}
    SCRIP = i[2]
    stock_name = i[1]
    stock_id =i[0]
    link = f'https://www.screener.in/company/{SCRIP}'
    hdr = {'User-Agent':'Mozilla/5.0'}
    req = Request(link,headers=hdr)
    column = ""
    try:
        page=urlopen(req)
        soup = BeautifulSoup(page)

        section_html = soup.find('section',{'id': 'cash-flow'})
        div_html = section_html.find('div',{'class': 'responsive-holder'})
        table_html = div_html.find('table',{'class': 'data-table'})
        thead_html = table_html.find('thead')
        trow_html = thead_html.find('tr')
        column = trow_html.text
        column = column.strip().split('\n\n')
        for i in range(0,len(column)):
            column[i] = column[i].strip()

        tbody_html = table_html.find('tbody')
        for trow_html in tbody_html.find_all("tr"):
            temp = []
            #trow_html = tbody_html.find('tr')
            ttitle_html = trow_html('td',{'class':'text'})

            title = trow_html.text
            title = title.strip().split('\n\n')
            header = title[0]
            header = header.replace('+','')
            header = header.replace('\n','')
#             print(header.strip()) 
            for i in range(1,len(title)):
                if title[i] != "":
                    temp.append(title[i].strip())
            stock[header.strip()] = temp
#         print(stock)
        print('---------------------------------------------------------------------------')
        print(stock_name)
        df =pd.DataFrame (stock, columns = list(stock.keys()),index = column)
        print(df)
        
        for j in range(0,len(df['Cash from Operating Activity'])):
            if 'Cash from Operating Activity' in df: cashflow['Cash from Operating Activity'] = df['Cash from Operating Activity'][j]
            if 'Cash from Investing Activity' in df: cashflow['Cash from Investing Activity'] = df['Cash from Investing Activity'][j]
            if 'Cash from Financing Activity' in df: cashflow['Cash from Financing Activity'] = df['Cash from Financing Activity'][j]
            if 'Net Cash Flow' in df: cashflow['Net Cash Flow'] = df['Net Cash Flow'][j]
            cur.execute("INSERT INTO cash_flow(stock_id,stock_name,stock_symbol,year_month,cash_from_operating_activity,cash_from_investing_activity,cash_from_financing_activity,net_cash_flow) VALUES("+str(stock_id)+",'"+stock_name+"','"+SCRIP+"','"+column[j]+"','"+cashflow['Cash from Operating Activity']+"','"+cashflow['Cash from Investing Activity']+"','"+cashflow['Cash from Financing Activity']+"','"+cashflow['Net Cash Flow']+"')")

    except:
        print(f'EXCEPTION THROWN: UNABLE TO FETCH DATA FOR {SCRIP}')


---------------------------------------------------------------------------
Tata Consultancy Services Ltd
         Cash from Operating Activity Cash from Investing Activity  \
Mar 2011                        5,720                         -830   
Mar 2012                        3,206                          433   
Mar 2013                        9,157                       -3,496   
Mar 2014                       12,942                       -7,143   
Mar 2015                       16,320                          586   
Mar 2016                       17,986                       -4,478   
Mar 2017                       23,132                      -15,834   
Mar 2018                       21,587                        5,728   
Mar 2019                       23,998                        5,876   
Mar 2020                       26,603                       12,967   
Mar 2021                       33,822                       -4,539   
Mar 2022                       36,127                 

---------------------------------------------------------------------------
JSW Steel Ltd
         Cash from Operating Activity Cash from Investing Activity  \
Mar 2011                        2,043                       -7,160   
Mar 2012                        2,139                       -3,928   
Mar 2013                        3,925                       -4,451   
Mar 2014                        5,274                       -4,687   
Mar 2015                        8,466                       -6,275   
Mar 2016                        5,670                       -5,033   
Mar 2017                        8,561                       -6,284   
Mar 2018                       12,174                       -6,134   
Mar 2019                       15,578                      -11,432   
Mar 2020                       14,110                      -19,092   
Mar 2021                       17,557                       -3,759   
Mar 2022                       23,335                      -17,652   


---------------------------------------------------------------------------
Infosys Ltd
         Cash from Operating Activity Cash from Investing Activity  \
Mar 2011                        4,275                        3,235   
Mar 2012                        5,921                          769   
Mar 2013                        6,942                       -2,779   
Mar 2014                        9,148                       -2,272   
Mar 2015                        7,955                          628   
Mar 2016                        9,578                       -1,216   
Mar 2017                       10,478                      -13,533   
Mar 2018                       12,475                        5,678   
Mar 2019                       13,989                         -637   
Mar 2020                       15,572                         -170   
Mar 2021                       19,902                       -6,286   
Mar 2022                       22,096                       -3,163   

 

In [184]:
#cur.execute("INSERT INTO cash_flow(stock_id,stock_name,stock_symbol,year_month,cash_from_operating_activity,cash_from_investing_activity,cash_from_financing_activity,net_cash_flow) VALUES("+str(stock_id)+",'"+stock_name+"','"+SCRIP+"','"+column[j]+"',"+cashflow['Cash from Operating Activity']+","+cashflow['Cash from Investing Activity']+","+cashflow['Cash from Financing Activity']+","+cashflow['Net Cash Flow']+")")

## Load Data in Quarterly_result Table

In [205]:
cur.execute("SELECT stock_id,stock_name,stock_symbol FROM stock_fundamental")
one_row = cur.fetchall()

for i in one_row:
    quarters = {'sales':0,'expenses':0,'operating_profit':0,'opm':0,'other':0,'interest':0,'depreciation':0,'profit':0,'tax':0,'net':0,'eps':0}
    stock = {}
    SCRIP = i[2]
    stock_name = i[1]
    stock_id =i[0]
    link = f'https://www.screener.in/company/{SCRIP}'
    hdr = {'User-Agent':'Mozilla/5.0'}
    req = Request(link,headers=hdr)
    stock ={}
    try:
        page=urlopen(req)
        soup = BeautifulSoup(page)

        section_html = soup.find('section',{'id': 'quarters'})
        div_html = section_html.find('div',{'class': 'responsive-holder fill-card-width'})
        table_html = div_html.find('table',{'class': 'data-table'})
        thead_html = table_html.find('thead')
        trow_html = thead_html.find('tr')
        column = trow_html.text
        column = column.strip().split('\n\n')
        for i in range(0,len(column)):
            column[i] = column[i].strip()
        #print("hello " ,column)

        tbody_html = table_html.find('tbody')
        for trow_html in tbody_html.find_all("tr"):
            temp = []
            #trow_html = tbody_html.find('tr')
            ttitle_html = trow_html('td',{'class':'text'})
            title = trow_html.text
            title = title.replace('%','')
            title = title.strip().split('\n\n')
            header = title[0].split()[0]
            for i in range(1,len(title)):
                if title[i] != "":
                    temp.append(title[i].strip())
            stock[header] = temp

        #print(stock)
        del stock['Raw']
        df =pd.DataFrame (stock, columns = list(stock.keys()),index = column)
        print(df)
        for j in range(0,len(df['Sales'])):
            if 'Sales' in df: quarters['sales'] = df['Sales'][j]
            if 'Expenses' in df: quarters['expenses'] = df['Expenses'][j]
            if 'Operating' in df: quarters['operating_profit'] = df['Operating'][j]
            if 'OPM' in df: quarters['opm'] = df['OPM'][j]
            if 'Other' in df: quarters['other'] = df['Other'][j]
            if 'Interest' in df: quarters['interest'] = df['Interest'][j]
            if 'Depreciation' in df: quarters['depreciation'] = df['Depreciation'][j]
            if 'Profit' in df: quarters['profit'] = df['Profit'][j]
            if 'Tax' in df: quarters['tax'] = df['Tax'][j]
            if 'Net' in df: quarters['net'] = df['Net'][j]
            if 'Eps' in df: quarters['eps'] = df['Eps'][j]
            cur.execute("INSERT INTO quarterly_result(stock_id,stock_name,stock_symbol,quarter_year,sales,expenses,operating_profit,opm,other,interest,depreciation,profit,tax,net,eps) VALUES("+str(stock_id)+",'"+str(stock_name)+"','"+str(SCRIP)+"','"+str(column[j])+"','"+str(quarters['sales'])+"','"+str(quarters['expenses'])+"','"+str(quarters['operating_profit'])+"','"+str(quarters['opm'])+"','"+str(quarters['other'])+"','"+str(quarters['interest'])+"','"+str(quarters['depreciation'])+"','"+str(quarters['profit'])+"','"+str(quarters['tax'])+"','"+str(quarters['net'])+"','"+str(quarters['eps'])+"')")

    except:
        print(f'EXCEPTION THROWN: UNABLE TO FETCH DATA FOR {SCRIP}')

            Sales Expenses Operating OPM  Other Interest Depreciation  Profit  \
Dec 2019   86,529   73,418    13,111  15  3,714    2,520        2,551  11,754   
Mar 2020   74,184   62,613    11,571  16   -527    4,161        2,685   4,198   
Jun 2020   47,461   40,350     7,111  15  8,661    5,251        2,175   8,346   
Sep 2020   56,287   48,781     7,506  13  4,164    4,819        2,233   4,618   
Dec 2020   61,263   52,573     8,690  14  3,500    3,140        2,209   6,841   
Mar 2021   80,656   70,463    10,193  13  2,797    3,001        2,582   7,407   
Jun 2021   86,147   74,687    11,460  13  3,588    2,351        2,509  10,188   
Sep 2021   96,925   84,548    12,377  13  3,769    2,487        2,705  10,954   
Dec 2021  111,152   97,257    13,895  12  3,443    2,301        2,597  12,440   
Mar 2022  129,479  114,898    14,581  11  3,072    1,984        2,465  13,204   
Jun 2022  147,534  125,526    22,008  15  2,531    2,616        2,251  19,672   
Sep 2022  137,550  125,563  

In [206]:
#  cur.execute("INSERT INTO quarterly_result(stock_id,stock_name,stock_symbol,quarter_year,sales,expenses,operating_profit,opm,other,interest,depreciation,profit,tax,net,eps) VALUES("+str(stock_id)+",'"+stock_name+"','"+SCRIP+"','"+column[j]+"','"+quarters['sales'][j]+"','"+quarters['expenses'][j]+"','"+quarters['operating_profit'][j]+"','"+quarters['opm']+"','"+quarters['other']+"','"+quarters['interest']+"','"+quarters['depreciation']+"','"+quarters['profit']+"','"+quarters['tax']+"','"+quarters['net']+"',"+quarters['eps']+")")

## Load Data in Balance_sheet Table

In [ ]:
cur.execute("SELECT stock_id,stock_name,stock_symbol FROM stock_fundamental")
one_row = cur.fetchall()

for i in one_row:
    stock = {}
    SCRIP = i[2]
    stock_name = i[1]
    stock_id =i[0]
    link = f'https://www.screener.in/company/{SCRIP}'
    hdr = {'User-Agent':'Mozilla/5.0'}
    req = Request(link,headers=hdr)
    balancesheet ={'share_capital':0,'reserves':0,'borrowings':0,'other_liabilities':0,'total_liabilities':0,'fixed_assets':0,'cwip':0,'investments':0,'other_assets':0,'total_assets':0}

    try:
        page=urlopen(req)
        soup = BeautifulSoup(page)

        section_html = soup.find('section',{'id': 'balance-sheet'})
        div_html = section_html.find('div',{'class': 'responsive-holder'})
        table_html = div_html.find('table',{'class': 'data-table'})
        thead_html = table_html.find('thead')
        trow_html = thead_html.find('tr')
        column = trow_html.text
        column = column.strip().split('\n\n')
        for i in range(0,len(column)):
            column[i] = column[i].strip()
        #print("hello " ,column)

        tbody_html = table_html.find('tbody')
        for trow_html in tbody_html.find_all("tr"):
            temp = []
            #trow_html = tbody_html.find('tr')
            ttitle_html = trow_html('td',{'class':'text'})

            title = trow_html.text
            title = title.strip().split('\n\n')
            header = title[0].strip()
            header = header.replace('+','')
            header = header.replace('\n','')
            header = header.replace('\xa0','')
            #print(header) 
            for i in range(1,len(title)):
                if title[i] != "":
                    temp.append(title[i].strip())
            stock[header] = temp
    #     print(stock)
        df =pd.DataFrame (stock, columns = list(stock.keys()),index = column)
        print(df)
        for j in range(0,len(df['Share Capital'])):
            if 'Share Capital' in df: balancesheet['share_capital'] = df['Share Capital'][j]
            if 'Borrowings' in df: balancesheet['borrowings'] = df['Borrowings'][j]
            if 'Reserves' in df: balancesheet['reserves'] = df['Reserves'][j]
            if 'Other Liabilities' in df: balancesheet['other_liabilities'] = df['Other Liabilities'][j]
            if 'Total Liabilities' in df: balancesheet['total_liabilities'] = df['Total Liabilities'][j]
            if 'Fixed Assets' in df: balancesheet['fixed_assets'] = df['Fixed Assets'][j]
            if 'CWIP' in df: balancesheet['cwip'] = df['CWIP'][j]
            if 'Investments' in df: balancesheet['investments'] = df['Investments'][j]
            if 'Other Assets' in df: balancesheet['other_assets'] = df['Other Assets'][j]
            if 'Total Assets' in df: balancesheet['total_assets'] = df['Total Assets'][j]
            cur.execute("INSERT INTO balance_sheet(stock_id,stock_name,stock_symbol,fin_month_year,share_capital,reserves,borrowings,other_liabilities,total_liabilities,fixed_assets,cwip,investments,other_assets,total_assets) VALUES("+str(stock_id)+",'"+str(stock_name)+"','"+str(SCRIP)+"','"+str(column[j])+"','"+str(balancesheet['share_capital'])+"','"+str(balancesheet['reserves'])+"','"+str(balancesheet['borrowings'])+"','"+str(balancesheet['other_liabilities'])+"','"+str(balancesheet['total_liabilities'])+"','"+str(balancesheet['fixed_assets'])+"','"+str(balancesheet['cwip'])+"','"+str(balancesheet['investments'])+"','"+str(balancesheet['other_assets'])+"','"+str(balancesheet['total_assets'])+"')")
           
    except:
        print(f'EXCEPTION THROWN: UNABLE TO FETCH DATA FOR {SCRIP}')

         Share Capital Reserves Borrowings Other Liabilities  \
Mar 2011           296   19,284         41             6,470   
Mar 2012           296   24,561        104             9,280   
Mar 2013           296   32,267        172            10,052   
Mar 2014           196   43,856        115            13,210   
Mar 2015           196   45,221        273            17,105   
Mar 2016           197   64,816        178            11,860   
Mar 2017           197   77,825        250            11,172   
Mar 2018           191   75,675        225            14,541   
Mar 2019           375   78,523         39            20,224   
Mar 2020           375   73,993      5,262            24,998   
Mar 2021           370   74,424      5,912            28,310   
Mar 2022           366   76,807      5,855            38,106   
Sep 2022           366   84,102      5,582            37,010   

         Total Liabilities Fixed Assets   CWIP Investments Other Assets  \
Mar 2011            25,991  

         Share Capital Reserves Borrowings Other Liabilities  \
Mar 2011           959   45,808     28,301            14,484   
Mar 2012           971   51,245     26,172            17,414   
Mar 2013           971   54,238     27,508            19,159   
Mar 2014           971   60,177     27,917            21,975   
Mar 2015           971   65,692     28,198            21,001   
Mar 2016           971   47,941     30,844            25,359   
Mar 2017           971   48,688     28,285            33,522   
Mar 2018         1,146   62,644     28,126            33,199   
Mar 2019         1,146   71,584     29,701            35,067   
Mar 2020         1,146   75,692     41,423            32,132   
Mar 2021         1,199   93,986     37,065            48,240   
Mar 2022         1,222  124,211     36,525            60,028   
Sep 2022         1,222  126,825     45,735            60,341   

         Total Liabilities Fixed Assets    CWIP Investments Other Assets  \
Mar 2011            89,552 

         Share Capital Reserves Borrowings Other Liabilities  \
Mar 2011         3,273  148,277     67,603            65,566   
Mar 2012         3,271  162,826     68,447            60,596   
Mar 2013         3,229  176,791     72,427            66,064   
Mar 2014         3,232  193,859     89,968            80,524   
Mar 2015         3,236  212,940     97,620            83,989   
Mar 2016         3,240  250,758    107,104           120,572   
Mar 2017         3,251  285,062    107,446           150,987   
Mar 2018         6,335  308,312    116,881           185,997   
Mar 2019         6,339  398,983    161,720           208,703   
Mar 2020         6,339  384,876    298,599           281,885   
Mar 2021         6,445  468,038    224,683           174,507   
Mar 2022         6,765  464,762    197,439           209,708   
Sep 2022         6,766  474,686    214,806           192,803   

         Total Liabilities Fixed Assets     CWIP Investments Other Assets  \
Mar 2011           284,719

In [ ]:
cur.execute("INSERT INTO balance_sheet(stock_id,stock_name,stock_symbol,fin_month_year,share_capital,reserves,borrowings,other_liabilities,total_liabilities,fixed_assets,cwip,investments,other_assets,total_assets) VALUES("+str(stock_id)+",'"+str(stock_name)+"','"+str(SCRIP)+"','"+str(column[j])+"','"+str(balancesheet['share_capital'])+"','"+str(balancesheet['reserves'])+"','"+str(balancesheet['borrowings'])+"','"+str(balancesheet['other_liabilities'])+"','"+str(balancesheet['total_liabilities'])+"','"+str(balancesheet['fixed_assets'])+"','"+str(balancesheet['cwip'])+"','"+str(balancesheet['investments'])+"','"+str(balancesheet['other_assets'])+"','"+str(balancesheet['total_assets'])+"')")